In [154]:
%load_ext autoreload

%autoreload 2

from scraper import Scraper
import pandas as pd

from fycharts.SpotifyCharts import SpotifyCharts
import sqlalchemy
import threading
from collections import defaultdict


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
tops = pd.read_pickle('data/dataframes/tops.pkl')
reg = pd.read_pickle('data/dataframes/regional.pkl')
vir = pd.read_pickle('data/dataframes/viral.pkl')

In [143]:
def countryTrackDates():
#     country 
#         "top" > track ID > date > # streams
#.        "viral" > [dates]

    groups = tops.groupby(['id', 'track_id'])
    d = defaultdict(dict)
    
    for (reg, tid), group in groups:
        group = group.drop(columns=['id', 'track_id'])
        d[reg][tid] = {}

        date_groups = group.groupby(['date'])
        for date, date_group in date_groups:
            date_group = date_group.drop(columns=['date'])
            d[reg][tid][date]=date_group.to_dict(orient='records')[0]['streams']
    
    v_groups = tops.groupby(['id', 'v_track_id'])
    vd = defaultdict(dict)
    for (reg, vid), group in v_groups:
        if vid:
            vd[reg][vid]=list(group.date.unique())
    vd = dict(vd)
    
    all_data = {}

    for key in d.keys():
        all_data[key]={'top': d[key], 'viral': vd[key]}

    return all_data


def countryPositionInfo():
    #Construct global position tables
    reg_globs = reg[reg.id == 'global']
    vir_globs = vir[vir.id == 'global']
    
    global_positions = {'top': {}, 'viral': {}}

    reg_global_groups = reg_globs.groupby(['date', 'track_id'])
    vir_global_groups = vir_globs.groupby(['date', 'track_id'])

    for (date, tid), group in reg_global_groups:
        global_positions['top'][(date, tid)] = group.iloc[0].Position

    for (date, tid), group in vir_global_groups:
        global_positions['viral'][(date, tid)] = group.iloc[0].Position  
        
    
    number_ones = reg[reg.Position==1]
    groups = number_ones.groupby(['date', 'track_id'])
    #get viral #1 tracks that weren't caught by #1 regional filter
    vir_number_ones = vir[vir.Position==1]
    vir_groups = vir_number_ones.groupby(['date', 'track_id'])
        
    data = defaultdict(dict)

    for (date, tid), group in groups:
        countries_on_top = len(group[group.id != 'global'].id.unique())#might need to drop global somewhere before
        reg_pos = global_positions['top'].get((date,tid), 201)
        vir_pos = global_positions['viral'].get((date,tid), 201)

        data[date][tid]= {'top_regions': countries_on_top, 
                          'position': {'top': reg_pos, 'viral': vir_pos}, 
                          'viral_regions': 0}



    for (date, tid), group in vir_groups:
        countries_on_top = len(group[group.id != 'global'].id.unique())


        existing = data[date].get(tid, None)
        if existing:
            existing['viral_regions']=countries_on_top
            data[date][tid]=existing
        else:
            reg_pos = global_positions['top'].get((date,tid), 201)
            vir_pos = global_positions['viral'].get((date,tid), 201)
            data[date][tid]= {'top_regions': 0, 
                          'global_position': {'top': reg_pos, 'viral': vir_pos}, 
                          'viral_regions': countries_on_top}
            
    return dict(data)


In [155]:
sc = Scraper()
sc.exportHighlights()

TypeError: Object of type 'int64' is not JSON serializable

In [151]:
d2

{'2019-01-01': {'0E9ZjEAyAwOXZ7wJC0PD33': {'top_regions': 1,
   'position': {'top': 28, 'viral': 201},
   'viral_regions': 0},
  '0cONHWTIfgEpAuVwbbwGgc': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '0sSG83UxY15ft1E2N9THYF': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '1YC7llsHtz0ScDtiLEVHZG': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '1ftBvRj542ol7LT1A0EtL7': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '1khmgu0pveJbkbpbkyvcQv': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '24mpJgP2YZUBif9NvmYyh4': {'top_regions': 1,
   'position': {'top': 201, 'viral': 201},
   'viral_regions': 0},
  '25sgk305KZfyuqVBQIahim': {'top_regions': 4,
   'position': {'top': 8, 'viral': 19},
   'viral_regions': 0},
  '2VxeLyX666F8uXCJ0dZF8B': {'top_regions': 2,
   'position': {'top': 23, 'vir

In [158]:
delta = pd.datetime.now() - pd.to_datetime('2019-07-20')

In [160]:
delta.days

372